# Notebook to access to data on CCDVisits for the System-level Science Performance Verification Sprint February 3-5 2025

- Confluence page : https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/372867091/System-level+Science+Performance+Verification+Sprint
- slides : https://docs.google.com/presentation/d/1bPjS5NGtiEL2lfjmvP1UYdG_tMEDfZxX26ayhf7MhtY/edit#slide=id.ga2f7fb1a1f_0_70

- where to find the campains
- https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs

- plot Navigator
- https://usdf-rsp.slac.stanford.edu/plot-navigator
- https://usdf-rsp.slac.stanford.edu/plot-navigator/plot/%2Frepo%2Fmain/LSSTComCam%2Fruns%2FDRP%2FDP1%2Fw_2025_05%2FDM-48666/objectTableCore_coaddInputCount_SkyPlot

- Notebooks examples
- https://github.com/lsst-dm/DMTR-401/blob/main/notebooks/test_LVV-T40_T1240.ipynb
- https://github.com/lsst-dm/DMTR-412/blob/tickets/DM-38728/notebooks/test_LVV-T1751_AM1_AM2.ipynb

- author : Sylvie Dagoret-Campagne
- creattion date : 2025-02-13
- Redo Visits like here : https://github.com/sylvielsstfr/LSST-Rehearsal2024/blob/main/notebooks/Visits/stat_on_visits_LSSTComCamSim.ipynb
- To find what I did on LSSTComCamSim : https://github.com/sylvielsstfr/LSST-Rehearsal2024/blob/main/notebooks/LightCurves/MultiColor_lightCurves-DMRehearsal2024_01-AuxTel-DZPOnCCD.ipynb
- **Confluence page** : https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/443613290/Science+Verification+Sprint+Feb+2025


In [ ]:
# Confirm that the version of the Science Pipelines is recent:
! echo $HOSTNAME
! eups list -s | grep lsst_distrib

In [ ]:
from lsst.daf.butler import Butler
import lsst.geom as geom

In [ ]:
import os
import gc
import glob
import numpy as np
import pandas as pd
import random

import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"  # Reselect Data Release 3, default
Gaia.ROW_LIMIT = 100000
from astropy.visualization import (MinMaxInterval, AsinhStretch, ZScaleInterval, LogStretch, LinearStretch,
                                   ImageNormalize)

In [ ]:
import seaborn as sns
from itertools import cycle, islice

In [ ]:
from astropy.time import Time
from datetime import datetime, timedelta

In [ ]:
import lsst.geom as geom
from lsst.geom import Angle
from lsst.geom import SpherePoint
from lsst.geom import AngleUnit

In [ ]:
# Set plotting defaults
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from matplotlib.colors import ListedColormap
from matplotlib import colors
zscale = ZScaleInterval()

# Set up some plotting defaults:
plt.rcParams.update({'figure.figsize' : (12, 8)})
plt.rcParams.update({'font.size' : 24})
plt.rcParams.update({'axes.linewidth' : 3})
plt.rcParams.update({'axes.labelweight' : 3})
plt.rcParams.update({'axes.titleweight' : 5})
plt.rcParams.update({'ytick.major.width' : 3})
plt.rcParams.update({'ytick.minor.width' : 2})
plt.rcParams.update({'ytick.major.size' : 8})
plt.rcParams.update({'ytick.minor.size' : 5})
plt.rcParams.update({'xtick.major.size' : 8})
plt.rcParams.update({'xtick.minor.size' : 5})
plt.rcParams.update({'xtick.major.width' : 3})
plt.rcParams.update({'xtick.minor.width' : 2})
plt.rcParams.update({'xtick.direction' : 'in'})
plt.rcParams.update({'ytick.direction' : 'in'})


In [ ]:
def angle_in_range(alpha, lower, upper):
    return (alpha - lower) % 360 <= (upper - lower) % 360

In [ ]:
def get_bbox_radec(wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    ramin,ramax,decmin,decmax in decimal degrees
    """

    xmin = bbox.beginX
    xmax = bbox.endX
    ymin = bbox.beginY
    ymax = bbox.endY
    
    radec_ll = wcs.pixelToSky(xmin, ymin)
    radec_ur = wcs.pixelToSky(xmax, ymax)
        
    return radec_ur.getRa().asDegrees(),radec_ll.getRa().asDegrees(), radec_ll.getDec().asDegrees(),radec_ur.getDec().asDegrees() 


In [ ]:
def isradec_inbbox_angularsep(ra,dec,wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    ra: ra in degree
    dec: dec in degree
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    float in degree
    
    """

   
    RAMIN,RAMAX,DECMIN,DECMAX = get_bbox_radec(wcs, bbox)
  
    RAMEAN = np.mean([RAMIN,RAMAX])
    DECMEAN = np.mean([DECMIN,DECMAX])

    sp0 = SpherePoint(longitude=geom.Angle(ra,geom.degrees),latitude=geom.Angle(dec,geom.degrees))
    sp1 = SpherePoint(longitude=geom.Angle(RAMEAN,geom.degrees),latitude=geom.Angle(DECMEAN,geom.degrees)) 
    
    sep = sp0.separation(sp1).asDegrees()
     
    return sep


In [ ]:
def isradec_inbbox_radec(ra,dec,wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    ra: ra in degree
    dec: dec in degree
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    Bool
    
    """

    RAMIN,RAMAX,DECMIN,DECMAX = get_bbox_radec(wcs, bbox)

    flag_ra = angle_in_range(ra,RAMIN,RAMAX)
    flag_dec = angle_in_range(dec,DECMIN,DECMAX)

    flag = flag_ra and flag_dec
    return flag


In [ ]:
def FindTractAndPatch(row):
    """
    Apply this function on ccdvisitTable dataframe to find the tract and patch for each visit
    """
     
    try:
        ra = row["ra"]
        dec = row["dec"]
        selectFlag = False
        # loop on tract
        for tractID in tractsId_list:
            tractInfo = skymap.generateTract(tractID)
            patches_selected = [patch.getSequentialIndex() for patch in tractInfo]
            wcs=tractInfo.getWcs()
            # loop on patches
            for patch in tractInfo:
                patchID = patch.getSequentialIndex()
                if patchID in patches_selected:
                    ibb=patch.getInnerBBox()
                    flag = isradec_inbbox_radec(ra,dec,wcs, ibb)
                    selectFlag =  selectFlag or flag
                    if selectFlag:
                        return pd.Series([tractID,patchID])
        return pd.Series([0,0])           
                
    except Exception as inst:
        print(type(inst))    # the exception type
        print(inst.args)     # arguments stored in .args
        print(inst)          # __str__ allows args to be printed directly,
        return pd.Series([0,0])  
        

In [ ]:
def FindTractAndPatchFromAngularSep(row):
    """
    Apply this function on ccdvisitTable dataframe to find the tract and patch for each visit
    Need the tractsId_list not to scan the whole skymap
    """

     
    try:
        ra = row["ra"]
        dec = row["dec"]
        
        # loop on tracts
        List_of_tractids = []
        List_of_patchids = []
        List_of_sep = []
        
        for tractID in tractsId_list:
            tractInfo = skymap.generateTract(tractID)
            patches_selected = [patch.getSequentialIndex() for patch in tractInfo]
         
            wcs=tractInfo.getWcs()
            # loop on patches
            patches_ids = []
            patches_sep = []
            # loop on patches
            for patch in tractInfo:
                patchID = patch.getSequentialIndex()
                if patchID in patches_selected:
                    ibb=patch.getInnerBBox()
                  
                    sep = isradec_inbbox_angularsep(ra,dec,wcs, ibb)
                    patches_ids.append(patchID)
                    patches_sep.append(sep)
                    
            # find the patch with the minimum distance
            patches_ids=np.array(patches_ids)
            patches_sep= np.array(patches_sep)
            idx_sepmin = int(np.where(patches_sep==patches_sep.min())[0])
            
            
            List_of_tractids.append(tractID)
            List_of_patchids.append(patches_ids[idx_sepmin])
            List_of_sep.append(patches_sep[idx_sepmin])
            
        List_of_tractids=np.array(List_of_tractids)
        List_of_patchids=np.array(List_of_patchids)
        List_of_sep= np.array(List_of_sep)
        idx_sepmin = int(np.where(List_of_sep==List_of_sep.min())[0])

        tractID_sel = List_of_tractids[idx_sepmin]
        patchID_sel = List_of_patchids[idx_sepmin]
       
            
        return pd.Series([tractID_sel,patchID_sel])
               
                
    except Exception as inst:
        print(type(inst))    # the exception type
        print(inst.args)     # arguments stored in .args
        print(inst)          # __str__ allows args to be printed directly,
        return pd.Series([0,0])  
        

In [ ]:
def plotAVisit(row,ax=None):
    """
    row : dataframe row
    ax 
    """

    if ax == None:
        fig, ax = plt.subplots(1,1,figsize=(5,5))
        
    visitid = row["visitId"]
    ra= row["ra"]
    dec = row["dec"]
    tract_id  = row["tractID"]
    patch_id  = row["patchID"]
    label = f"tract {tract_id} , patch {patch_id}"              
    
    tractInfo = skymap.generateTract(tract_id)
    patches_selected = [patch.getSequentialIndex() for patch in tractInfo]     
    wcs=tractInfo.getWcs()
            
    # loop on patches
    all_ra = []
    all_dec = []
    corners = []
    for patch in tractInfo:
        patchID = patch.getSequentialIndex()
        if patchID == patch_id :
            ibb=patch.getInnerBBox()
            RAMIN,RAMAX,DECMIN,DECMAX = get_bbox_radec(wcs, ibb)
            for icorn,corner in enumerate(ibb.getCorners()):
                p = geom.Point2D(corner.getX(), corner.getY())
                coord = wcs.pixelToSky(p)
                corners.append([coord.getRa().asDegrees(), coord.getDec().asDegrees()])
                all_ra.append(coord.getRa().asDegrees()) 
                all_dec.append(coord.getDec().asDegrees()) 
    if (len(all_ra)>0) and (len(all_dec)>0):
        all_ra.append(all_ra[0])
        all_dec.append(all_dec[0])
        print(ax)
        ax.plot(all_ra,all_dec,'b-',lw=3,label=label)
        ax.scatter([ra],[dec],marker = 'o',s=20,c="r",label=visitid)
        ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
        ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
        ax.legend()
                



## Start Here with initialisation


- Check here the collection available : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs

In [ ]:
# The output repo is tagged with the Jira ticket number "DM-40356":
repo = '/repo/main'
collection = 'LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666'
instrument = "LSSTComCam"
skymapName = "lsst_cells_v1"
where_clause = "instrument = \'" + instrument+ "\'"
collectionStr = collection.replace("/", "_")
NDET = 9
fn_ccdVisit_tracts_patches = f"ccdVisittractpatch_{collectionStr}.csv"

In [ ]:
# Initialize the butler repo:
butler = Butler(repo, collections=collection)
registry = butler.registry

In [ ]:
skymap = butler.get('skyMap', skymap=skymapName, collections=collection)

In [ ]:
# Check here the collections available
#for _ in registry.queryCollections():
#    if "LSSTComCam/runs/DRP/DP1/w_2025_05" in _:
#        print(_)

In [ ]:
# List a number of usefull data-product to explore
if 0:
    for datasetType in registry.queryDatasetTypes():
        if registry.queryDatasets(datasetType, collections=collection).any(
            execute=False, exact=False
        ):
            # Limit search results to the data products
            if (
                ("_config" not in datasetType.name)
                and ("_log" not in datasetType.name)
                and ("_metadata" not in datasetType.name)
                and ("_resource_usage" not in datasetType.name)
                and (("Table" in datasetType.name) or ("Zeropointp" in datasetType.name) or ("fgcm" in datasetType.name) or ("transm" in datasetType.name) or ("Transm" in datasetType.name) )
            ):
                print(datasetType)

## Get list of Tracts and Patches

In [ ]:
datasettype = "objectTable_tract"
therefs = butler.registry.queryDatasets(datasettype,  collections=collection)

In [ ]:
tractsId_list = np.unique([ref.dataId['tract'] for ref in therefs])
tractsId_list = sorted(tractsId_list)
print(tractsId_list)

In [ ]:
dict_tractpatches = {}
for ref in therefs:
    tract = ref.dataId["tract"]
    table = butler.get(ref) 
    list_of_patches = table['patch'].unique()
    dict_tractpatches[tract] = list_of_patches

## Get CCD Visits

In [ ]:
datasettype = "ccdVisitTable"
ccd_visit_table = butler.get(datasettype,collections=collection)

In [ ]:
# Compute nightobs
ccd_visit_table["nightObs"] = ccd_visit_table.apply(lambda x: x['visitId']//100_000, axis=1)

In [ ]:
#  compute time
ccd_visit_table["Time"] = pd.to_datetime(ccd_visit_table['obsStart'])

In [ ]:
# get airmass
ccd_visit_table["airmass"] = ccd_visit_table["zenithDistance"].apply(lambda x: 1/np.cos(np.pi/180.*x))

In [ ]:
# Short visit ID
ccd_visit_table["ccdVid"] = ccd_visit_table.index -  ccd_visit_table.index[0]
ccd_visit_table["Vid"] = ccd_visit_table["visitId"]  -   ccd_visit_table["visitId"].iloc[0]

In [ ]:
ccd_visit_table

### Associate Tract and Patch for each visit

To associate Tract and Patch to every visit is a long procedure. This association is done at the first pass time in this notebook
and written into a csv file. At next execution, this visitid-tract-patch is read into a pandas dtaframe. The association is done
on a pair of columns ((visitId,detector).

In [ ]:
# to speed up the calculation of tractID, patchID per visit, this file is saved
# or read back
if os.path.isfile(fn_ccdVisit_tracts_patches):
    print(f"{fn_ccdVisit_tracts_patches} found !!! ==> Read it !!!" )
    ccd_visit_tract_patch_table = pd.read_csv(fn_ccdVisit_tracts_patches,index_col=0) 
    # need to add only "tractID","patchID" to ccd_visit_table
    ccd_visit_tract_patch_table_only = ccd_visit_tract_patch_table[["visitId","detector","tractID","patchID"]]
    # do the merging on the columns (visitId,detector), does not add suffixes to column names
    new_df = pd.merge(
    left=ccd_visit_table, 
    right=ccd_visit_tract_patch_table_only,
    how='left',
    left_on=['visitId', 'detector'],
    right_on=['visitId', 'detector'], suffixes=('', ''))
    #Overwrite the ccd_visit_table by the merge result
    new_df
    ccd_visit_table=new_df
else:
    print(f"{fn_ccdVisit_tracts_patches} NOT found !!! ==> Create it !!! " )
    ccd_visit_table[["tractID","patchID"]] = ccd_visit_table.apply(FindTractAndPatch, axis=1,result_type ='expand')
    # check if some (tract,patch) search failed
    ccd_visit_table_patchnotfound = ccd_visit_table[ccd_visit_table.patchID==0]
    if len(ccd_visit_table_patchnotfound)>0:
        ccd_visit_table_patchnotfound.drop(columns=["tractID","patchID"],inplace=True)
        ccd_visit_table_patchnotfound[["tractID","patchID"]] = ccd_visit_table_patchnotfound.apply(FindTractAndPatchFromAngularSep, axis=1,result_type ='expand')
        for visitindex, row in ccd_visit_table_patchnotfound.iterrows(): 
            ccd_visit_table.loc[visitindex,["tractID","patchID"]] = ccd_visit_table_patchnotfound.loc[visitindex,["tractID","patchID"]]
    #save a subsample for the visit 
    columns_selected = ["visitId","band","detector","ra","dec","llcra","llcdec","ulcra","ulcdec","urcra","urcdec","lrcra","lrcdec","ccdVid","Vid","nightObs","tractID","patchID","zeroPoint","airmass","skyBg","skyNoise","expTime"]
    ccd_visit_tract_patch_table = ccd_visit_table[columns_selected]
    ccd_visit_tract_patch_table.to_csv(fn_ccdVisit_tracts_patches) 


In [ ]:
ccd_visit_table.head()

## Plot One visit

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,6))
plotAVisit(ccd_visit_table.iloc[0],ax=ax)
ax.set_aspect('equal')
ax.set_title(f"{instrument} \n {collectionStr}",fontsize=12)

## 3) Statistics

### 3.1) Statistics per detectors and per filter

In [ ]:
ser_ccdvisit_perdetectorband_size = ccd_visit_table.groupby(["detector","band"]).size()

In [ ]:
ser_ccdvisit_perdetectorband_size

In [ ]:
df_ccdvisit_perdetectorband = ser_ccdvisit_perdetectorband_size.unstack(level=1)
df_ccdvisit_perdetectorband 

In [ ]:
# retorder filters
df_ccdvisit_perdetectorband = df_ccdvisit_perdetectorband[["u","g","r","i","z","y"]]

In [ ]:
my_colors = list(islice(cycle(['b','g', 'r', 'orange','grey','purple']), None, len(df_ccdvisit_perdetectorband)))

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(16,6))
df_ccdvisit_perdetectorband.plot.bar(color=my_colors,ax=ax,rot=0,title=f"{instrument} : Nb of ccdvisits per detector and per filter") 
ax.legend(bbox_to_anchor=(1.1, 1.05))
plt.suptitle(f"{instrument} , {collectionStr}",fontsize=12)

In [ ]:
cmap = ListedColormap(sns.color_palette("hls", NDET))
all_det_colors = [cmap.colors[idx] for idx in range(NDET)]

In [ ]:
df_ccdvisit_perdetectorband = ser_ccdvisit_perdetectorband_size.unstack(level=0)
df_ccdvisit_perdetectorband 

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(16,6))
df_ccdvisit_perdetectorband.plot.bar(ax=ax,color=all_det_colors,rot=0,title=f"{instrument} : Number of CCDvisits per filter and per detector")
#ax.legend(loc="upper right")
ax.legend(bbox_to_anchor=(1.1, 1.05))
plt.suptitle(f"{instrument} , {collectionStr}",fontsize=12)

In [ ]:
my_colors = list(islice(cycle(['b','g', 'r', 'orange','grey','purple']), None, NDET))
fig,ax = plt.subplots(1,1,figsize=(18,6))
ser_ccdvisit_perdetectorband_size.plot.bar(x="detector",color=my_colors,title=f"{instrument} : Number of ccdvisits per detector and per filter")
plt.suptitle(f"{instrument} , {collectionStr}",fontsize=12)

### 3.2) Statistics on Visits per tract

In [ ]:
ser_tract_det_size = ccd_visit_table[["Vid","ccdVid","detector","tractID","patchID"]].groupby(["tractID","detector"]).size()

In [ ]:
ser_tract_det_size

In [ ]:
df_ccdvisit_pertractdet = ser_tract_det_size.unstack(level=1)

In [ ]:
df_ccdvisit_pertractdet

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,6))
df_ccdvisit_pertractdet.plot.bar(ax=ax,color=all_det_colors,title=f"{instrument} :Number of ccdvisits per patch per detector",grid=True) 
ax.legend(bbox_to_anchor=(1.1, 1.05))
plt.suptitle(f"{instrument} , {collectionStr}",fontsize=12)

## Vid and TractID

In [ ]:
List_OfTracts = df_ccdvisit_pertractdet.index
NTracts = len(List_OfTracts)-1
NTracts

In [ ]:
cmap = ListedColormap(sns.color_palette("hls", NTracts))
list_tracts_colors = [cmap.colors[idx] for idx in range(NTracts)]

In [ ]:
list_tracts_colors = [ colors.to_rgb('k') ] + list_tracts_colors

In [ ]:
dict_tract_colors = {}
for idx,itr in enumerate(List_OfTracts):
    dict_tract_colors[itr] = list_tracts_colors[idx]
    

In [ ]:
all_colors = [ dict_tract_colors[itr] for itr in ccd_visit_table["tractID"].values]

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,6))
ccd_visit_table.plot.scatter(y="tractID",x ="visitId",ax=ax,marker='o',color=all_colors,title=f"{instrument} : tractID vs visit sequence number",grid=True)
plt.suptitle(f"{instrument} , {collectionStr}",fontsize=12)

In [ ]:
ccd_visit_table["Time"]

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,6))
ccd_visit_table.plot.scatter(y="tractID",x ="Time",ax=ax,marker='o',rot=45,color=all_colors,title=f"{instrument} : tractID vs date",grid=True)
plt.suptitle(f"{instrument} , {collectionStr}",fontsize=12)

### Statistics per night and per tracts

In [ ]:
df_stat_pertract = ccd_visit_table.groupby(by=["nightObs","tractID"]).count()["visitId"]
df_stat_pertract

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,6))
df_stat_pertract.unstack(level=1).plot(kind="bar",ax=ax)
ax.legend(bbox_to_anchor=(1.1, 1.05),ncols=2)
ax.set_title(f"{instrument} : number of visits per nightobs pertracts")
plt.suptitle(f"{instrument} , {collectionStr}",fontsize=12)
plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,6))
df_stat_pertract.unstack(level=0).plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(16,5),grid=True)
ax.legend(bbox_to_anchor=(1.1, 1.05),ncols=2)
ax.set_title(f"{instrument} : number of visits per tracts per night")
plt.suptitle(f"{instrument} , {collectionStr}",fontsize=12)
plt.show()

## Number of observations in each filters

In [ ]:
ccd_visit_table[ccd_visit_table["band"] == "u"]

In [ ]:
df =  ccd_visit_table
ccd_visit_table_counts = df.groupby(['nightObs','band']).count()["visitId"]
ccd_visit_table_counts_u = df[df["band"] == "u"].groupby(['nightObs']).count()["visitId"]
ccd_visit_table_counts_g = df[df["band"] == "g"].groupby(['nightObs']).count()["visitId"]
ccd_visit_table_counts_r = df[df["band"] == "r"].groupby(['nightObs']).count()["visitId"]
ccd_visit_table_counts_i = df[df["band"] == "i"].groupby(['nightObs']).count()["visitId"]
ccd_visit_table_counts_z = df[df["band"] == "z"].groupby(['nightObs']).count()["visitId"]
ccd_visit_table_counts_y = df[df["band"] == "y"].groupby(['nightObs']).count()["visitId"]

In [ ]:
def convertNumToDatestr(num):
    year = num//10_000
    month= (num-year*10_000)//100
    day = (num-year*10_000-month*100)

    year_str = str(year).zfill(4)
    month_str = str(month).zfill(2)
    day_str = str(day).zfill(2)
    
    datestr = f"{year_str}-{month_str}-{day_str}"
    return pd.to_datetime(datestr)

In [ ]:
dt_u = pd.to_datetime([convertNumToDatestr(num) for num in ccd_visit_table_counts_u.index])
dt_g = pd.to_datetime([convertNumToDatestr(num) for num in ccd_visit_table_counts_g.index])
dt_r = pd.to_datetime([convertNumToDatestr(num) for num in ccd_visit_table_counts_r.index ])
dt_i = pd.to_datetime([convertNumToDatestr(num) for num in ccd_visit_table_counts_i.index ])
dt_z = pd.to_datetime([convertNumToDatestr(num) for num in ccd_visit_table_counts_z.index ])
dt_y = pd.to_datetime([convertNumToDatestr(num) for num in ccd_visit_table_counts_y.index ])

In [ ]:
n_u = ccd_visit_table_counts_u.values
n_g = ccd_visit_table_counts_g.values
n_r = ccd_visit_table_counts_r.values
n_i = ccd_visit_table_counts_i.values
n_z = ccd_visit_table_counts_z.values
n_y = ccd_visit_table_counts_y.values


In [ ]:
n_u_tot = np.sum(n_u)
n_g_tot = np.sum(n_g)
n_r_tot = np.sum(n_r)
n_i_tot = np.sum(n_i)
n_z_tot = np.sum(n_z)
n_y_tot = np.sum(n_y)

n_u_min = np.min(n_u)
n_g_min = np.min(n_g)
n_r_min = np.min(n_r)
n_i_min = np.min(n_i)
n_z_min = np.min(n_z)
n_y_min = np.min(n_y)

n_u_max = np.max(n_u)
n_g_max = np.max(n_g)
n_r_max = np.max(n_r)
n_i_max = np.max(n_i)
n_z_max = np.max(n_z)
n_y_max = np.max(n_y)

In [ ]:
s_u = n_u.cumsum()
s_g = n_g.cumsum()
s_r = n_r.cumsum()
s_i = n_i.cumsum()
s_z = n_z.cumsum()
s_y = n_y.cumsum()

In [ ]:
import matplotlib.dates as mdates
month_locator = mdates.MonthLocator(interval=1)
day_locator = mdates.DayLocator(interval=1)
year_month_formatter = mdates.DateFormatter("%Y-%m-%d") 
fig, axs = plt.subplots(6,1,figsize=(16, 16),sharex=True,layout="constrained")

ax0, ax1,ax2,ax3,ax4,ax5 = axs

# fit 1
#ax.plot(dt,df_spec_visit.values,marker='o', markersize=15,lw=0)
ax0.bar(dt_u,n_u,lw=5,facecolor='b')
ax0.xaxis.set_major_locator(month_locator) # Locator for major axis only.
ax0.xaxis.set_minor_locator(day_locator)
ax0.xaxis.set_major_formatter(year_month_formatter)
ax0.grid()

title0 = f"number of phot obs per night in U"
#ax0.set_title(title0)


# fit 1
#ax.plot(dt,df_spec_visit.values,marker='o', markersize=15,lw=0)
ax1.bar(dt_g,n_g,lw=5,facecolor='g')
ax1.xaxis.set_major_locator(month_locator) # Locator for major axis only.
ax1.xaxis.set_minor_locator(day_locator)
ax1.xaxis.set_major_formatter(year_month_formatter)
ax1.grid()

title1 = f"number of phot obs per night in G"
#ax1.set_title(title1)

#fig.autofmt_xdate() # rotation
ax1.set_xticks(ax1.get_xticks(), ax1.get_xticklabels(), rotation=45, ha='right')

#ax2.bar(dt,df_spec_visit.cumsum().values,lw=5,facecolor='b')
ax2.bar(dt_r,n_r,lw=5,facecolor='r')
ax2.xaxis.set_major_locator(month_locator) # Locator for major axis only.
ax2.xaxis.set_minor_locator(day_locator)
ax2.xaxis.set_major_formatter(year_month_formatter)
ax2.grid()

#fig.autofmt_xdate() # rotation
ax2.set_xticks(ax2.get_xticks(), ax2.get_xticklabels(), rotation=45, ha='right')
title2 = f"number of phot obs per night R"
#ax2.set_title(title2)

#ax2.bar(dt,df_spec_visit.cumsum().values,lw=5,facecolor='b')
ax3.bar(dt_i,n_i,lw=5,facecolor='orange')
ax3.xaxis.set_major_locator(month_locator) # Locator for major axis only.
ax3.xaxis.set_minor_locator(day_locator)
ax3.xaxis.set_major_formatter(year_month_formatter)
ax3.grid()

#fig.autofmt_xdate() # rotation
ax3.set_xticks(ax3.get_xticks(), ax3.get_xticklabels(), rotation=45, ha='right')
title3 = f"number of phot obs per night in I"
#ax3.set_title(title3)

#ax2.bar(dt,df_spec_visit.cumsum().values,lw=5,facecolor='b')
ax4.bar(dt_z,n_z,lw=5,facecolor='grey')
ax4.xaxis.set_major_locator(month_locator) # Locator for major axis only.
ax4.xaxis.set_minor_locator(day_locator)
ax4.xaxis.set_major_formatter(year_month_formatter)
ax4.grid()

#fig.autofmt_xdate() # rotation
ax4.set_xticks(ax4.get_xticks(), ax4.get_xticklabels(), rotation=45, ha='right')
title4 = f"number of phot obs per night in Z"
#ax4.set_title(title4)


#ax2.bar(dt,df_spec_visit.cumsum().values,lw=5,facecolor='b')
ax5.bar(dt_y,n_y,lw=5,facecolor='k')
ax5.xaxis.set_major_locator(month_locator) # Locator for major axis only.
ax5.xaxis.set_major_locator(day_locator)
ax5.xaxis.set_major_formatter(year_month_formatter)
ax5.grid()

#fig.autofmt_xdate() # rotation
ax5.set_xticks(ax5.get_xticks(), ax5.get_xticklabels(), rotation=45, ha='right')
title5 = f"number of phot obs per night in Y"
#ax5.set_title(title5)


ax5.xaxis.set_major_locator(day_locator) #Every year
plt.locator_params(axis="x", nbins=10)

plt.show()



In [ ]:
import matplotlib.dates as mdates
month_locator = mdates.MonthLocator(interval=1)
day_locator = mdates.DayLocator(interval=1)
year_month_formatter = mdates.DateFormatter("%Y-%m") 
fig, axs = plt.subplots(2,1,figsize=(20, 16),sharex=True)

ax1,ax2 = axs

dt = timedelta(hours=10)

# fit 1
#ax.plot(dt,df_spec_visit.values,marker='o', markersize=15,lw=0)
ax1.bar(dt_u-dt,n_u,lw=5,facecolor='b',label="u")
ax1.bar(dt_g-dt,n_g,lw=5,facecolor='g',label="g")
ax1.bar(dt_r,n_r,lw=5,facecolor='r',label="r")
ax1.bar(dt_i+dt,n_i,lw=5,facecolor='orange',label="i")
ax1.bar(dt_z+2*dt,n_z,lw=5,facecolor='grey',label="z")
ax1.bar(dt_y+3*dt,n_y,lw=5,facecolor='k',label="y")
ax1.xaxis.set_major_locator(month_locator) # Locator for major axis only.
ax1.xaxis.set_minor_locator(day_locator)
ax1.xaxis.set_major_formatter(year_month_formatter)
ax1.grid()
ax1.legend()
ax1.set_xticks(ax1.get_xticks(), ax1.get_xticklabels(), rotation=45, ha='right')

title = f"number of phot obs per night"
ax1.set_title(title)

ax2.bar(dt_u-dt,s_u,lw=5,facecolor='b',label="u")
ax2.bar(dt_g-dt,s_g,lw=5,facecolor='g',label="g")
ax2.bar(dt_r,s_r,lw=5,facecolor='r',label="r")
ax2.bar(dt_i+dt,s_i,lw=5,facecolor='orange',label="i")
ax2.bar(dt_z+2*dt,s_z,lw=5,facecolor='grey',label="z")
ax2.bar(dt_y+3*dt,s_y,lw=5,facecolor='k',label="y")
ax2.xaxis.set_major_locator(month_locator) # Locator for major axis only.
ax2.xaxis.set_major_locator(day_locator)
ax2.xaxis.set_major_formatter(year_month_formatter)
ax2.grid()
ax2.legend()
ax2.set_xticks(ax2.get_xticks(), ax2.get_xticklabels(), rotation=45, ha='right')

title = f"cumulative number of phot obs per night (n_tot_g = {n_g_tot} , n_r_tot = {n_r_tot} , n_i_tot = {n_i_tot} ,  n_z_tot = {n_z_tot}, n_y_tot = {n_y_tot})"
ax2.set_title(title)


plt.tight_layout()

In [ ]:
ccd_visit_table.band == "b"

In [ ]:
ccdVisitTable_u = ccd_visit_table[ccd_visit_table.band== "u"]
ccdVisitTable_g = ccd_visit_table[ccd_visit_table.band== "g"]
ccdVisitTable_r = ccd_visit_table[ccd_visit_table.band== "r"]
ccdVisitTable_i = ccd_visit_table[ccd_visit_table.band== "i"]
ccdVisitTable_z = ccd_visit_table[ccd_visit_table.band== "z"]
ccdVisitTable_y = ccd_visit_table[ccd_visit_table.band== "y"]

In [ ]:
ccdVisitTable_u["zeroPoint"]

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%d")

fig,axs = plt.subplots(4,1,figsize=(16,20),layout="constrained")
ax1,ax2,ax3,ax4 = axs
leg1 = ax1.get_legend()
leg2 = ax2.get_legend()
leg3 = ax3.get_legend()
leg4 = ax4.get_legend()

ccdVisitTable_u.plot(x="Time",y="zeroPoint",marker='+',c="b",lw=0.0,ax=ax1,grid=True,legend=leg1,label="u")
ccdVisitTable_g.plot(x="Time",y="zeroPoint",marker='+',c="g",lw=0.0,ax=ax1,grid=True,legend=leg1,label="g")
ccdVisitTable_r.plot(x="Time",y="zeroPoint",marker='+',c="r",lw=0.0,ax=ax1,grid=True,legend=leg1,label="r")
ccdVisitTable_i.plot(x="Time",y="zeroPoint",marker='+',c="orange",lw=0.0,ax=ax1,grid=True,legend=leg1,label="i")
ccdVisitTable_z.plot(x="Time",y="zeroPoint",marker='+',c="grey",lw=0.0,ax=ax1,grid=True,legend=leg1,label="z")
ccdVisitTable_g.plot(x="Time",y="zeroPoint",marker='+',c="k",lw=0.0,ax=ax1,grid=True,legend=leg1,label="y")
#ax1.set_title("zeroPoint")
ax1.set_ylabel("zeroPoint")
ax1.set_xlabel(None)
ax1.xaxis.set_major_formatter(date_form)
ax1.legend(bbox_to_anchor=(1.1, 1.05))
#ax1_up = ax1.secondary_xaxis("top", functions=( dt_to_mjd, mjd_to_dt))
#ccdVisitTable_g.plot(x="expMidptMJD",y="zeroPoint",marker='+',c="g",lw=0.0,ax=ax1_up,grid=True,label="g")


ccdVisitTable_u.plot(x="Time",y="airmass",marker='+',c="b",lw=0.0,ax=ax2,grid=True,legend=leg2,label="u")
ccdVisitTable_g.plot(x="Time",y="airmass",marker='+',c="g",lw=0.0,ax=ax2,grid=True,legend=leg2,label="g")
ccdVisitTable_r.plot(x="Time",y="airmass",marker='+',c="r",lw=0.0,ax=ax2,grid=True,legend=leg2,label="r")
ccdVisitTable_i.plot(x="Time",y="airmass",marker='+',c="orange",lw=0.0,ax=ax2,grid=True,legend=leg2,label="i")
ccdVisitTable_z.plot(x="Time",y="airmass",marker='+',c="grey",lw=0.0,ax=ax2,grid=True,legend=leg2,label="z")
ccdVisitTable_y.plot(x="Time",y="airmass",marker='+',c="k",lw=0.0,ax=ax2,grid=True,legend=leg2,label="y")
ax2.set_xlabel(None)
ax2.set_ylabel("airmass")
ax2.xaxis.set_major_formatter(date_form)
ax2.legend(bbox_to_anchor=(1.1, 1.05))


ccdVisitTable_u.plot(x="Time",y="seeing",marker='+',c="b",lw=0.0,ax=ax3,grid=True,legend=leg3,label="u")
ccdVisitTable_g.plot(x="Time",y="seeing",marker='+',c="g",lw=0.0,ax=ax3,grid=True,legend=leg3,label="g")
ccdVisitTable_r.plot(x="Time",y="seeing",marker='+',c="r",lw=0.0,ax=ax3,grid=True,legend=leg3,label="r")
ccdVisitTable_i.plot(x="Time",y="seeing",marker='+',c="orange",lw=0.0,ax=ax3,grid=True,legend=leg3,label="i")
ccdVisitTable_z.plot(x="Time",y="seeing",marker='+',c="grey",lw=0.0,ax=ax3,grid=True,legend=leg3,label="z")
ccdVisitTable_y.plot(x="Time",y="seeing",marker='+',c="k",lw=0.0,ax=ax3,grid=True,legend=leg3,label="y")
#ax2.set_title("airmass")
ax3.set_ylabel("seeing")
ax3.set_xlabel(None)
ax3.xaxis.set_major_formatter(date_form)
ax3.legend(bbox_to_anchor=(1.1, 1.05))


ccdVisitTable_u.plot(x="Time",y="magLim",marker='+',c="b",lw=0.0,ax=ax4,grid=True,legend=leg4,label="u")
ccdVisitTable_g.plot(x="Time",y="magLim",marker='+',c="g",lw=0.0,ax=ax4,grid=True,legend=leg4,label="g")
ccdVisitTable_r.plot(x="Time",y="magLim",marker='+',c="r",lw=0.0,ax=ax4,grid=True,legend=leg4,label="r")
ccdVisitTable_i.plot(x="Time",y="magLim",marker='+',c="orange",lw=0.0,ax=ax4,grid=True,legend=leg4,label="i")
ccdVisitTable_z.plot(x="Time",y="magLim",marker='+',c="grey",lw=0.0,ax=ax4,grid=True,legend=leg4,label="z")
ccdVisitTable_y.plot(x="Time",y="magLim",marker='+',c="k",lw=0.0,ax=ax4,grid=True,legend=leg4,label="y")

ax4.set_ylabel("magLim")
ax4.xaxis.set_major_formatter(date_form)
ax4.legend(bbox_to_anchor=(1.1, 1.05))
ax4.set_xlabel(None)


#df.plot(x="Time",y="coord_dec",marker='o',c="g",lw=0.0,ax=ax2,grid=True)
plt.suptitle(f"{instrument} , {collectionStr}",fontsize=12)
plt.show()


## Simple visit table

In [ ]:
visit_table = butler.get('visitTable', dataId={'instrument': instrument}, collections = collection)    
visit_table